In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h1> 1 - IOU(box, centroid)  metric should be used </h1>

In [ ]:
!pip install torchvision

import os
from IPython.display import display as dis
from IPython.display import Image as im
import cv2
import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import torch


from torchvision.ops import box_iou

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/util

In [ ]:
import shutil

shutil.copy("/content/drive/MyDrive/dataset_udacity/box_priors.json", "./")
shutil.copy("/content/drive/MyDrive/dataset_udacity/file_orderer.json", "./")

'./file_orderer.json'

In [39]:
from torch import nn
from collections import OrderedDict

class normed_Conv2d(nn.Conv2d):
  def __init__(self, **kwargs):
    super(normed_Conv2d, self).__init__(**kwargs)
    self.norm_layer = nn.BatchNorm2d(num_features = self.out_channels)
    self.activation = nn.SiLU()

  def forward(self, x):
    x = super(normed_Conv2d, self).forward(x)
    x = self.norm_layer(x)
    x = self.activation(x)
    return x




""" NN.SEQUENTIAL CAN BE USED INSTEAD OF MODULELIST, BUT IT TAKES THE SAME TIME AS LOOPING OVER LAYERS IN MODULELIST
DEFINITION SHOULD BE DONE BY MAKING AN ORDEREDDICT OF THE LAYERS"""
class bottleneck(nn.Module):
  def __init__(self,
               in_channels,
               num_1x1_filters_per_rep,
               num_3x3_filters_per_rep,
               num_reps,
               block_number = None,
               downsample = False
               ):
    super(bottleneck, self).__init__()

    self.num_1x1_filters_per_rep = num_1x1_filters_per_rep
    self.num_3x3_filters_per_rep = num_3x3_filters_per_rep
    self.num_reps = num_reps

    layer_block = nn.ModuleList()

    if block_number == None:
        block_number = "NA"

    if downsample:
      padding_3x3 = 0
    else:
      padding_3x3 = 1

    for rep_no in range(self.num_reps):

      rep = OrderedDict()

      rep[f"block_{block_number}_rep_{rep_no}_ConvBNSiLU_1x1"] = normed_Conv2d(
          in_channels =  in_channels,
          out_channels = self.num_1x1_filters_per_rep,
          kernel_size = 1,
          # padding = 0 #same
          )

      rep[f"block_{block_number}_rep_{rep_no}_ConvBNSiLU_3x3"] = normed_Conv2d(
          in_channels = self.num_1x1_filters_per_rep,
          out_channels = self.num_3x3_filters_per_rep,
          kernel_size = 3,
          padding = padding_3x3 # same
          )

      rep = nn.Sequential(rep)

      layer_block.add_module(name = f"block_{block_number}_rep_{rep_no}",
                             module = rep)

      in_channels = self.num_3x3_filters_per_rep

    self.layer_block = layer_block


  def forward(self, x):
    init_layer = True
    for layer in self.layer_block:
      if init_layer:
        x = layer(x)
      else:
        x = layer(x) + x

    return x



class backbone(nn.Module):
  def __init__(self):
    super(backbone, self).__init__()

    self.super_block = OrderedDict()

    ############### INPUT LAYER ################
    self.super_block["input_layer"] = normed_Conv2d(
        in_channels = 3,
        out_channels = 32,
        kernel_size = 3
    )
    ############################################

    ######## FIRST CONV ONLY BOTTLENECK ########
    self.super_block["ConvBNSiLU_1"] = normed_Conv2d(
        in_channels = 32,
        out_channels = 32,
        kernel_size = 3
    )

    self.super_block["ConvBSSiLU_2"] = normed_Conv2d(
        in_channels = 32,
        out_channels = 64,
        kernel_size = 3,
        stride = 2,
        padding = 1
    )
    ############################################

    #SEQUENTIAL REPETITVE BOTTLENECK SEQUENCE ##

    self.super_block["bottleneck_1"] = bottleneck(
        in_channels = 64,
        num_1x1_filters_per_rep = 32,
        num_3x3_filters_per_rep = 64,
        num_reps = 2,
        block_number = 0,
        downsample = True,
    )

    self.super_block["ConvBSSiLU_3"] = normed_Conv2d(
        in_channels = 64,
        out_channels = 128,
        kernel_size = 3,
        stride = 2,
        padding = 1
    )

    self.super_block["bottleneck_2"] = bottleneck(
        in_channels = 128,
        num_1x1_filters_per_rep = 64,
        num_3x3_filters_per_rep = 128,
        num_reps = 6,
        block_number = 1,
        downsample = True
    )

    self.super_block["ConvBNSiLU_4"] = normed_Conv2d(
        in_channels = 128,
        out_channels = 256,
        kernel_size = 3,
        stride = 2,
        padding = 1
    )

    self.super_block["bottleneck_3"] = bottleneck(
        in_channels = 256,
        num_1x1_filters_per_rep = 128,
        num_3x3_filters_per_rep = 256,
        num_reps = 8,
        block_number = 2,
        downsample = True

    )

    self.super_block["ConvBNSiLU_5"] = normed_Conv2d(
        in_channels = 256,
        out_channels = 512,
        kernel_size = 3,
        stride = 2,
        padding = 1,
    )

    self.super_block["bottleneck_4"] = bottleneck(
        in_channels = 512,
        num_3x3_filters_per_rep = 256,
        num_1x1_filters_per_rep = 512,
        num_reps = 8,
        block_number = 3,
        downsample = True
    )
    ##########################################

    self.super_block = nn.Sequential(self.super_block)

  def forward(self, x):
    x = self.super_block(x)

    return x





In [40]:
a = bottleneck_block()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
a.to(device)
start = time.time()
print(a(torch.rand([1,3,700,700]).to(device)).shape)
print(f"time taken to do a forward pass of the bottlenxk {time.time() - start}")

torch.Size([1, 256, 16, 16])
time taken to do a forward pass of the bottlenxk 0.17354655265808105


<h1> PLAYGROUD</H1>

In [64]:
def dim_calc(a, n1, n2, n3, n4, n5):
  a-= (2*n1)
  print(a)
  a/=2
  print(a)
  a-= (2*n2)
  print(a)
  a/=2
  print(a)
  a-= (2*n3)
  print(a)
  a/=2
  print(a)
  a-= (2*n4)
  print(a)
  a/=2
  print(a)
  a-= (2*n5)
  print(a)

  return a


def dim_del(x1,x2,x3,x4,x5):
  dim = (2*x1) + (2**2 * x2) + (2**3 * x3) + (2**4 * x4) + (2**5 * x5)

  return dim


def find_in_dim(out_dim , n1, n2, n3, n4, n5):
  out_dim += (2*n5)
  out_dim *= 2
  out_dim += (2*n4)
  out_dim *= 2
  out_dim += (2*n3)
  out_dim *= 2
  out_dim += (2*n2)
  out_dim *= 2
  out_dim += (2*n1)
  print(f"num_layer : {n1+n2+n3+n4+n5}")
  return out_dim

In [66]:
find_in_dim(16, 2, 2, 6, 8, 8)

num_layer : 26


700

In [67]:
dim_calc(700, 2, 2, 6, 8, 8)

696
348.0
344.0
172.0
160.0
80.0
64.0
32.0
16.0


16.0

In [6]:
import torch
torch.device("cuda")

device(type='cuda')

In [22]:
import torch
import time

a = bottleneck(in_channels = 128,
               num_1x1_filters_per_rep = 64,
               num_3x3_filters_per_rep = 128,
               num_reps = 6,
               block_number = 0,
               downsample = True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

a.to(device)

start = time.time()
result = a(torch.rand([5,128,172,172]).to(device))
print(result.shape)
end = time.time()

print(f"device : {device} : time taken : {end - start}")

torch.Size([5, 128, 160, 160])
device : cuda : time taken : 0.18221807479858398


In [55]:
# from torch import nn
import torch

b = nn.Conv2d(in_channels = 512,
              out_channels = 1024,
              kernel_size = 3,
              # padding = 1,
              # stride = 2
              )
# a(b(torch.rand([5,512,8,8]))).shape

b(torch.rand([5,512, 15, 15])).shape

torch.Size([5, 1024, 13, 13])

In [ ]:
a = seq_conv_block_1_3(
    in_channels = 100,
    num_1x1_filters_per_rep = 32,
    num_3x3_filters_per_rep = 64,
    num_reps = 800,
    block_number= 0
)

b = conv_block_1_3(
    in_channels = 100,
    num_1x1_filters_per_rep = 32,
    num_3x3_filters_per_rep = 64,
    num_reps = 800=00,
    block_number = 0
)

len(tuple(a.get_submodule("layer_block").named_children()))

In [ ]:
testmodule = conv_block_1_3(
    in_channels = 10,
    num_filters_1x1 = 32,
    num_filters_3x3 = 64,
    num_reps = 6,
    block_number = 10
)

print(tuple(testmodule.get_submodule("layer_block").named_children()))

print(testmodule(torch.ones([1,10,8,10])).shape)

(('block_10_rep_0_conv_1x1', Conv2d(10, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)), ('block_10_rep_0_conv_3x3', Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)), ('block_10_rep_1_conv_1x1', Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)), ('block_10_rep_1_conv_3x3', Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)), ('block_10_rep_2_conv_1x1', Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)), ('block_10_rep_2_conv_3x3', Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)), ('block_10_rep_3_conv_1x1', Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)), ('block_10_rep_3_conv_3x3', Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)), ('block_10_rep_4_conv_1x1', Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)), ('block_10_rep_4_conv_3x3', Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)), ('block_10_rep_5_conv_1x1', Conv2d(64, 32, kernel_size=(1, 

torch.Size([1, 64, 10, 10])

In [ ]:
from torch import nn

a = nn.ModuleList()

a.append(nn.Conv2d(10,10,3))
a.append(nn.Linear(10,10))
a.append(nn.Linear(10,10))
a.append(nn.Linear(10,9))
a.register_module(name = "mr conv 1",
                  module = nn.Conv2d(12,10,3))
a.register_module(name = "mr conv 1",
                  module = nn.Conv2d(9,10,3))  #using the same name will overwrite the module
a.register_module(name = "mr conv 2",
                  module = nn.Conv2d(9,11,3))
a.register_module(name = "mr conv 2",
                  module = nn.Conv2d(9,10,3))

a.append(nn.Conv2d(9,11,3))

tuple(a.named_children())


(('0', Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))),
 ('1', Linear(in_features=10, out_features=10, bias=True)),
 ('2', Linear(in_features=10, out_features=10, bias=True)),
 ('3', Linear(in_features=10, out_features=9, bias=True)),
 ('mr conv 1', Conv2d(9, 10, kernel_size=(3, 3), stride=(1, 1))),
 ('mr conv 2', Conv2d(9, 10, kernel_size=(3, 3), stride=(1, 1))),
 ('6', Conv2d(9, 11, kernel_size=(3, 3), stride=(1, 1))))